In [ ]:
import random
import math
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from operator import add
from operator import sub

In [ ]:
# Compute the root density
def compute_density(graph):
    Density = []
    for i in range(len(graph)):
        G = graph[i]
        r = []
        D = 0
        circle = 0
        # change circle to one node
        a = list(nx.simple_cycles(G))
        if a != []:
            circle = 1
            n = [x for x in range(8) if x not in a[0]] + [a[0][0]]
            #print(i)
            # pick the edges for the new combined node, we call it a[0][0]
            l_out = [G.out_edges(x) for x in a[0]]
            l_in = [G.in_edges(x) for x in a[0]]
            # keep the start to be a[0][0] and delete the edge to nonexist nodes.
            l1 = [(k,v) if (k == a[0][0]) else (a[0][0], v) for (k, v) in l_out[0]]
            l1 += [(k,v) if (k == a[0][0]) else (a[0][0], v) for (k, v) in l_out[1]] 
            l1 += [(k,v) if (k == a[0][0]) else (a[0][0], v) for (k, v) in l_out[2]]
            l1 = [(k,v) if (v not in a[0] and v!=a[0][0]) else () for (k, v) in l1]
            l1 = [t for t in l1 if t]
            # keep the end to be a[0][0] and delete the edge to nonexist nodes
            l2 = [(k,v) if (v == a[0][0]) else (k, a[0][0]) for (k, v) in l_in[0]]
            l2 += [(k,v) if (v == a[0][0]) else (k, a[0][0]) for (k, v) in l_in[1]]
            l2 += [(k,v) if (v == a[0][0]) else (k, a[0][0]) for (k, v) in l_in[2]]
            l2 = [(k,v) if (k not in a[0] and k!=a[0][0]) else () for (k, v) in l2]
            l2 = [t for t in l2 if t]
            
            l = l1 + l2
            
            G.remove_nodes_from(a[0])
            G.add_node(a[0][0])
            G.add_edges_from(l)
            
            for j1 in n:
                if len(G.reverse().neighbors(j1)) == 0:
                    r += [j1]
            #print([n,r])
        # calculate the mean D
        else:
            list_nodes = G.nodes()
            r = [val for val in list_nodes if len(G.reverse().neighbors(val)) == 0]
            
        if len(r) != 0:
            for k in r:
                aa = nx.single_source_shortest_path_length(G,k)
                #if len(aa) == 1:
                    #print(i,r,aa)
                D += sum(aa.values())/(len(aa)-1)
                #D.append(sum([nx.shortest_path_length(G,source=k,target=x) for x in lis])/m)
            
            D = D/len(r)
            #Density.append([np.mean(D),i])
        Density.append([D,len(r),circle])
    return Density

In [ ]:
# Compute the relative density
def relative_density(graph):
    Density = []
    for i in range(len(graph)):
        G = graph[i]
        # change circle to one node
        a = list(nx.simple_cycles(G))
        l1 = []
        l2 = []
        while a != []:
            n = [x for x in range(8) if x not in a[0]] + [a[0][0]]
    
            # pick the edges for the new combined node, we call it a[0][0]
            l_out = [G.out_edges(x) for x in a[0]]
            l_in = [G.in_edges(x) for x in a[0]]
            # calculate the len of l_out and l_in
            num_out = len(l_out)
            num_in = len(l_in)
            # keep the start to be a[0][0] and delete the edge to nonexist nodes.
            for ou in range(num_out):
                l1 += [(k,v) if (k == a[0][0]) else (a[0][0], v) for (k, v) in l_out[ou]]
                
            l1 = [(k,v) if (v not in a[0] and v!=a[0][0]) else () for (k, v) in l1]
            l1 = [t for t in l1 if t]    

            # keep the end to be a[0][0] and delete the edge to nonexist nodes
            for ip in range(num_in):
                l2 += [(k,v) if (v == a[0][0]) else (k, a[0][0]) for (k, v) in l_in[ip]]
                
            l2 = [(k,v) if (k not in a[0] and k!=a[0][0]) else () for (k, v) in l2]
            l2 = [t for t in l2 if t]
            
            l = l1 + l2
            
            G.remove_nodes_from(a[0])
            G.add_node(a[0][0])
            G.add_edges_from(l)
            a = list(nx.simple_cycles(G))
            
        
        path = nx.dag_longest_path(G)
        start = path[0]
        excep = []
        # collect the nodes that disconnect with start.
        excep = [val for val in G.nodes() if nx.has_path(G, start, val) == False]
        # collect the nodes that connect with start.
        leaf = [val for val in G.nodes() if nx.has_path(G, start, val) == True]
        leaf.remove(start)
        D = {start: 0}
        # assign the value for the main root
        for j in leaf:
            all_path_len = [len(x) for x in nx.all_simple_paths(G,start,j)]
            path_len = max(all_path_len)-1
            D.update({j:path_len})
        # collect the new root
        root_new = []
        for k in excep:
            if len(G.reverse().neighbors(k)) == 0:
                root_new += [k]
        
        while True:
            # assign value for the new root,collect rest_root that cannot be assign value
            rest_root = []
            update_root = []
            for r in root_new:
                lis_con = [val for val in list(D.keys()) if nx.has_path(G, r, val) == True]
                value = [] 
                if len(lis_con) != 0:
                    for con in lis_con:
                        all_path_len = [len(x) for x in nx.all_simple_paths(G,r,con)]
                        path_len = max(all_path_len)-1
                        value += [D.get(con)-path_len]
            
                    D.update({r:min(value)})
                    update_root += [r]
                else:
                    rest_root += [r]
            #assign value for the rest nodes
            rest_lis = [val for val in G.nodes() if val not in list(D.keys())]
            rest_lis = [val for val in rest_lis if val not in root_new]
    
            if rest_lis != []:
                for re in rest_lis:
                    b = [val for val in update_root if nx.has_path(G, val, re) == True]
                    all_path_len = [len(x) for x in nx.all_simple_paths(G,b[0],re)]
                    path_len = max(all_path_len)-1
                    assign_value = D.get(b[0]) + path_len
                    D.update({re:assign_value})
            
            #there are some root left
            if rest_root != []:
                root_new = rest_root
            else:
                break
        #Density.append(D)
        Density.append(sum(D.values())/len(D))
    return Density
                